In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.signal import butter, lfilter
from scipy.signal import freqz

In [16]:
subjectData = {}

In [17]:
ns = 10
for i in range(1,ns):
    data_path = os.path.join("A{:02d}T.npz".format(i))
    subject = 'subject{:02d}'.format(i)

    subjectData[subject] = np.load(data_path)

In [18]:
print(type(subjectData['subject01']))
print(subjectData['subject01'].files)

<class 'numpy.lib.npyio.NpzFile'>
['s', 'etyp', 'epos', 'edur', 'artifacts']


s: contains time-series recorded EEG signals of shape MxN array. N is the number of electrodes (22 EEG and 3 EOG), M can vary

etpye: event type to indicate event occurence

epos: event position denoting event start at s


edur: event duration artifacts: size of 288x1, 6x48 = 288 where 6 is the number of runs with 48 trials. 48 trials have 12 trials of 4 class each

In [19]:
print('Sample\t Electrodes')
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    print(subjectData[sub]['s'].shape)

Sample	 Electrodes
(672528, 25)
(677169, 25)
(660530, 25)
(600915, 25)
(686120, 25)
(678980, 25)
(681071, 25)
(675270, 25)
(673328, 25)


In [20]:
# Bandpass filter
def butter_bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut/nyq
    high = highcut/nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b,a,signal, axis=-1)
    return y

In [21]:
def butterFilter(cleanData, data, sub):
    lowcut = 4
    highcut = 40
    fs = 250
    print('Processing', sub)
    cleanData[sub] = butter_bandpass_filter(data, lowcut, highcut, fs)

In [22]:
# Transpose the raw EEG data to apply filtering on the time-series axis
rawData = {}
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    rawData[sub] = subjectData[sub]['s'].T

    print(rawData[sub].shape)

(25, 672528)
(25, 677169)
(25, 660530)
(25, 600915)
(25, 686120)
(25, 678980)
(25, 681071)
(25, 675270)
(25, 673328)


In [23]:
cleanEEG = {}
for i in range(1,ns):
    sub = 'subject{:02d}'.format(i)
    butterFilter(cleanEEG, rawData[sub], sub)

    print(cleanEEG[sub].shape)

Processing subject01
(25, 672528)
Processing subject02
(25, 677169)
Processing subject03
(25, 660530)
Processing subject04
(25, 600915)
Processing subject05
(25, 686120)
Processing subject06
(25, 678980)
Processing subject07
(25, 681071)
Processing subject08
(25, 675270)
Processing subject09
(25, 673328)


There are 4 classes with event code as 769, 770, 771, 772 for left, right, foot and tongue.

In [24]:
left_class_code = 769
right_class_code = 770
foot_class_code = 771
tongue_class_code = 772

for i in range(1,2):
    sub = 'subject{:02d}'.format(i)
    left = subjectData[sub]['epos'][subjectData[sub]['etyp'] == left_class_code]
    right = subjectData[sub]['epos'][subjectData[sub]['etyp'] == right_class_code]
    foot = subjectData[sub]['epos'][subjectData[sub]['etyp'] == foot_class_code]
    tongue = subjectData[sub]['epos'][subjectData[sub]['etyp'] == tongue_class_code]



In [25]:
left

array([ 98242, 100250, 112163, 114058, 116030, 124069, 126135, 130237,
       140283, 154531, 168776, 186876, 189204, 191207, 197085, 201113,
       208998, 214809, 229142, 239079, 245162, 251366, 261676, 283711,
       289960, 297948, 299998, 313611, 321841, 333953, 346094, 348201,
       354427, 364371, 376373, 380546, 384877, 386795, 398764, 402668,
       404563, 430788, 442929, 447128, 451262, 455346, 459281, 469374,
       481712, 489701, 491618, 505314, 529584, 537693, 541871, 546054,
       548097, 550161, 552181, 572157, 576544, 600205, 622505, 626419,
       632502, 636599, 642889, 646996, 654876, 663044, 666878, 671051])

These are the sample points where the event has started

Steps to start with

1. Remove EOG channels from all subjects

2. Crop the data for each subject such that you get the data during the motor imagery peroid (refer Experimental_Paradigm)


In [40]:
print(subjectData['subject02'].files)

print(len(subjectData['subject02']['s']))
print(len(subjectData['subject02']['etyp']))
print(len(subjectData['subject02']['epos']))
print(len(subjectData['subject02']['edur']))
print((subjectData['subject02']['etyp']))


['s', 'etyp', 'epos', 'edur', 'artifacts']
677169
606
606
606
[[32766]
 [  276]
 [32766]
 [  277]
 [32766]
 [ 1072]
 [32766]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  769]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  770]
 [  768]
 [  769]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  770]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  771]
 [  768]
 [  771]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [  768]
 [  772]
 [32766]
 [